In [7]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import os

In [12]:
def orb_bf_match_and_find(template_img, test_img, template_img_filename, test_img_filename, min_matches, object_name): 
    # Scale train_img to be the same size as query_img
    test_img = cv.resize(test_img, (template_img.shape[1], template_img.shape[0]))
    # Create an ORB object
    orb = cv.ORB_create()
    
    features1, des1 = orb.detectAndCompute(template_img, None)
    features2, des2 = orb.detectAndCompute(test_img, None)

    # Create Brute-Force matcher object
    bf = cv.BFMatcher(cv.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = template_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        
        test_img = cv.polylines(test_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)

        if object_name in test_img_filename:
            print('TP')
        else:
            print('FP')
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
        if object_name in test_img_filename:
            print('FN')
        else:
            print('TN')
    print('Total matches found: {}'.format(len(good)))        
    result_img = cv.drawMatchesKnn(template_img, features1, test_img, features2, good, None, flags = 4)
    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/orb/bf/{}_and_{}.jpg'.format(template_img_filename, test_img_filename), result_img)

def orb_bf_match_with_images_folder(path, min_matches, object_name):
    template = cv.imread(path)
    template_filename = os.path.splitext(os.path.basename(path))[0]
    for filename in os.listdir('output/orb/bf'):
        if filename.startswith(template_filename):
            os.remove('output/orb/bf/' + filename)
    for filename in os.listdir('images'):
        test_img = cv.imread('images/' + filename)
        test_img_filename = os.path.splitext(filename)[0]
        orb_bf_match_and_find(template, test_img, template_filename, test_img_filename, min_matches, object_name)

In [13]:
# Running matching with remote control template
min_matches = 10
orb_bf_match_with_images_folder('templates/remote-1.jpg', min_matches, 'remote')
orb_bf_match_with_images_folder('templates/remote-2.jpg', min_matches, 'remote')

Not enough good matches are found - 8/10
TN
Total matches found: 8
TP
Total matches found: 17
Not enough good matches are found - 8/10
TN
Total matches found: 8
TP
Total matches found: 12
TP
Total matches found: 21
Not enough good matches are found - 8/10
TN
Total matches found: 8
Not enough good matches are found - 8/10
TN
Total matches found: 8
Not enough good matches are found - 3/10
TN
Total matches found: 3
TP
Total matches found: 49
TP
Total matches found: 16
TP
Total matches found: 29
Not enough good matches are found - 9/10
TN
Total matches found: 9
Not enough good matches are found - 9/10
TN
Total matches found: 9
TP
Total matches found: 24
Not enough good matches are found - 5/10
TN
Total matches found: 5
TP
Total matches found: 15
TP
Total matches found: 14
Not enough good matches are found - 2/10
TN
Total matches found: 2
Not enough good matches are found - 5/10
TN
Total matches found: 5
Not enough good matches are found - 3/10
TN
Total matches found: 3
TP
Total matches fou

In [10]:
# Running matching with earphone template
min_matches = 25
orb_bf_match_with_images_folder('templates/earphone-1.jpg', min_matches, 'earphone')
orb_bf_match_with_images_folder('templates/earphone-2.jpg', min_matches, 'earphone')

Not enough good matches are found - 23/25
TN
Total matches found: 23
FP
Total matches found: 27
Not enough good matches are found - 17/25
TN
Total matches found: 17
Not enough good matches are found - 20/25
FN
Total matches found: 20
Not enough good matches are found - 11/25
FN
Total matches found: 11
Not enough good matches are found - 13/25
TN
Total matches found: 13
Not enough good matches are found - 18/25
TN
Total matches found: 18
FP
Total matches found: 31
Not enough good matches are found - 12/25
TN
Total matches found: 12
TP
Total matches found: 40
Not enough good matches are found - 11/25
FN
Total matches found: 11
Not enough good matches are found - 19/25
TN
Total matches found: 19
Not enough good matches are found - 20/25
TN
Total matches found: 20
FP
Total matches found: 31
Not enough good matches are found - 14/25
TN
Total matches found: 14
TP
Total matches found: 26
Not enough good matches are found - 16/25
FN
Total matches found: 16
Not enough good matches are found - 1

In [11]:
# Running matching with elephant template
min_matches = 8
orb_bf_match_with_images_folder('templates/elephant-1.jpg', min_matches, 'elephant')
orb_bf_match_with_images_folder('templates/elephant-2.jpg', min_matches, 'elephant')

FP
Total matches found: 11
TP
Total matches found: 15
Not enough good matches are found - 4/8
TN
Total matches found: 4
Not enough good matches are found - 7/8
FN
Total matches found: 7
TP
Total matches found: 8
FP
Total matches found: 8
Not enough good matches are found - 7/8
TN
Total matches found: 7
FP
Total matches found: 9
TP
Total matches found: 8
Not enough good matches are found - 7/8
FN
Total matches found: 7
Not enough good matches are found - 6/8
FN
Total matches found: 6
Not enough good matches are found - 2/8
TN
Total matches found: 2
Not enough good matches are found - 6/8
TN
Total matches found: 6
Not enough good matches are found - 6/8
FN
Total matches found: 6
Not enough good matches are found - 4/8
TN
Total matches found: 4
Not enough good matches are found - 4/8
FN
Total matches found: 4
Not enough good matches are found - 6/8
FN
Total matches found: 6
Not enough good matches are found - 2/8
TN
Total matches found: 2
Not enough good matches are found - 7/8
TN
Total m

In [14]:
def orb_flann_match_and_find(template_img, test_img, template_img_filename, test_img_filename, min_matches, object_name): 
    # Scale train_img to be the same size as query_img
    test_img = cv.resize(test_img, (template_img.shape[1], template_img.shape[0]))
    # Create an ORB object
    orb = cv.ORB_create()
    
    features1, des1 = orb.detectAndCompute(template_img, None)
    features2, des2 = orb.detectAndCompute(test_img, None)

    # Create a FLANN matcher object
    index_params = dict(algorithm = 6, table_number = 6, key_size = 12, multi_probe_level = 1)
    search_params = dict()

    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = template_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv.perspectiveTransform(pts, M)
        test_img = cv.polylines(test_img, [np.int32(dst)], True, (0, 255, 0), 2, cv.LINE_AA)
        
        if object_name in test_img_filename:
            print('TP')
        else:
            print('FP')
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
        if object_name in test_img_filename:
            print('FN')
        else:
            print('TN')
    print('Total matches found: {}'.format(len(good)))        
    result_img = cv.drawMatchesKnn(template_img, features1, test_img, features2, good, None, flags = 4)

    cv.putText(result_img, 'Total matches found: {}'.format(len(good)), (200, 200), cv.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 255), 2, cv.LINE_AA)
    cv.imwrite('output/orb/flann/{}_and_{}.jpg'.format(template_img_filename, test_img_filename), result_img)

def orb_flann_match_with_images_folder(path, min_matches, object_name):
    template = cv.imread(path)
    template_filename = os.path.splitext(os.path.basename(path))[0]
    for filename in os.listdir('output/orb/flann'):
        if filename.startswith(template_filename):
            os.remove('output/orb/flann/' + filename)
    for filename in os.listdir('images'):
        test_img = cv.imread('images/' + filename)
        test_img_filename = os.path.splitext(filename)[0]
        orb_bf_match_and_find(template, test_img, template_filename, test_img_filename, min_matches, object_name)

In [17]:
# Running matching with remote control template
min_matches = 10
orb_flann_match_with_images_folder('templates/remote-1.jpg', min_matches)
orb_flann_match_with_images_folder('templates/remote-2.jpg', min_matches)

Total matches found: 55
Total matches found: 30
Total matches found: 21
Total matches found: 65
Total matches found: 18
Total matches found: 38
Total matches found: 27
Total matches found: 28
Total matches found: 21
Total matches found: 14
Total matches found: 24
Total matches found: 37
Total matches found: 38
Total matches found: 26
Total matches found: 21
Total matches found: 47
Total matches found: 14
Total matches found: 36
Total matches found: 24
Total matches found: 27
Total matches found: 11
Not enough good matches are found - 9/10
Total matches found: 9
Total matches found: 22
Total matches found: 44


In [ ]:
# Running matching with earphone template
min_matches = 25
orb_flann_match_with_images_folder('templates/earphone-1.jpg', min_matches, 'earphone')
orb_flann_match_with_images_folder('templates/earphone-2.jpg', min_matches, 'earphone')

In [ ]:
# Running matching with elephant template
min_matches = 8
orb_flann_match_with_images_folder('templates/elephant-1.jpg', min_matches, 'elephant')
orb_flann_match_with_images_folder('templates/elephant-2.jpg', min_matches, 'elephant')